First, install scanpy using `mamba install -c conda-forge scanpy`

In [1]:
import anndata
import scanpy as sc
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

In [2]:
download_base = Path('/data/scRNA/ABCA/AIBS/AWS/')
abc_cache = AbcProjectCache.from_cache_dir(download_base)

In [3]:
cell_metadata = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='cell_metadata_with_cluster_annotation'
)
cell_metadata.set_index('cell_label', inplace=True)

/home/momo/miniforge3/envs/abca-env/lib/python3.10/site-packages/abc_atlas_access/abc_atlas_cache/abc_project_cache.py:429: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path, **kwargs)


In [1]:
cell_metadata.head()

NameError: name 'cell_metadata' is not defined

In [5]:
expr_path = '/data/scRNA/ABCA/AIBS/AWS/expression_matrices/WMB-10Xv3/20230630/WMB-10Xv3-CTXsp-log2.h5ad'
expression_data = sc.read_h5ad(expr_path)
expression_data

AnnData object with n_obs × n_vars = 78464 × 32285
    obs: 'cell_barcode', 'library_label', 'anatomical_division_label'
    var: 'gene_symbol'
    uns: 'normalization', 'parent', 'parent_layer', 'parent_rows'

In [6]:
expression_data.obs.head()

,cell_barcode,library_label,anatomical_division_label
cell_label,,,
GCCCGAAGTCAGTTTG-344_C04,GCCCGAAGTCAGTTTG,L8TX_200827_01_E10,CTXsp
AGGTTACTCGCGCCAA-332_B01,AGGTTACTCGCGCCAA,L8TX_200820_01_D01,CTXsp
ATGAGGGAGTGTTCCA-332_A01,ATGAGGGAGTGTTCCA,L8TX_200820_01_E02,CTXsp
CATAGACAGTCTAACC-299_B02,CATAGACAGTCTAACC,L8TX_200723_01_G09,CTXsp
GAGTTACCATGACGAG-299_A02,GAGTTACCATGACGAG,L8TX_200723_01_F09,CTXsp


In [7]:
expression_data.obs_names

Index(['GCCCGAAGTCAGTTTG-344_C04', 'AGGTTACTCGCGCCAA-332_B01',
       'ATGAGGGAGTGTTCCA-332_A01', 'CATAGACAGTCTAACC-299_B02',
       'GAGTTACCATGACGAG-299_A02', 'GGAGCAATCTCTCGAC-299_A02',
       'GGCTTGGTCTCCGAGG-332_B01', 'TCGAAGTGTTCGGTCG-308_B03',
       'GAGACTTAGTGCCCGT-308_A03', 'AGCCAATAGTAGCCAG-344_B04',
       ...
       'TTTGGTTTCAGGACGA-344_C04', 'TTTGGTTTCCCAGTGG-323_A02',
       'TTTGGTTTCCGCACGA-308_A03', 'TTTGTTGAGAGCACTG-332_C01',
       'TTTGTTGCACTGGACC-341_A01', 'TTTGTTGGTCCCTAAA-344_A04',
       'TTTGTTGGTGAGCGAT-308_C03', 'TTTGTTGGTTGTGGCC-308_C03',
       'TTTGTTGTCTAAGCCA-332_C01', 'TTTGTTGTCTTACGGA-341_A01'],
      dtype='object', name='cell_label', length=78464)

In [8]:
# Align metadata with expression data
common_cells = expression_data.obs_names.intersection(cell_metadata.index)
expression_data = expression_data[common_cells, :]

# Add metadata to the AnnData object
expression_data.obs = cell_metadata.loc[common_cells]
expression_data

AnnData object with n_obs × n_vars = 78223 × 32285
    obs: 'cell_barcode', 'barcoded_cell_sample_label', 'library_label', 'feature_matrix_label', 'entity', 'brain_section_label', 'library_method', 'region_of_interest_acronym', 'donor_label', 'donor_genotype', 'donor_sex', 'dataset_label', 'x', 'y', 'cluster_alias', 'neurotransmitter', 'class', 'subclass', 'supertype', 'cluster', 'neurotransmitter_color', 'class_color', 'subclass_color', 'supertype_color', 'cluster_color', 'region_of_interest_order', 'region_of_interest_color'
    var: 'gene_symbol'
    uns: 'normalization', 'parent', 'parent_layer', 'parent_rows'

In [9]:
expression_data.obs.head()

,cell_barcode,barcoded_cell_sample_label,library_label,feature_matrix_label,entity,brain_section_label,library_method,region_of_interest_acronym,donor_label,donor_genotype,...,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,region_of_interest_order,region_of_interest_color
cell_label,,,,,,,,,,,,,,,,,,,,,
GCCCGAAGTCAGTTTG-344_C04,GCCCGAAGTCAGTTTG,344_C04,L8TX_200827_01_E10,WMB-10Xv3-CTXsp,cell,NaN,10Xv3,CTXsp,Snap25-IRES2-Cre;Ai14-539605,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,079 CEA-BST Six3 Cyp26b1 Gaba,0367 CEA-BST Six3 Cyp26b1 Gaba_1,1327 CEA-BST Six3 Cyp26b1 Gaba_1,#FF3358,#450099,#73C1FF,#660F30,#FFED73,17,#994817
AGGTTACTCGCGCCAA-332_B01,AGGTTACTCGCGCCAA,332_B01,L8TX_200820_01_D01,WMB-10Xv3-CTXsp,cell,NaN,10Xv3,CTXsp,Snap25-IRES2-Cre;Ai14-537272,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,079 CEA-BST Six3 Cyp26b1 Gaba,0367 CEA-BST Six3 Cyp26b1 Gaba_1,1330 CEA-BST Six3 Cyp26b1 Gaba_1,#FF3358,#450099,#73C1FF,#660F30,#814DFF,17,#994817
ATGAGGGAGTGTTCCA-332_A01,ATGAGGGAGTGTTCCA,332_A01,L8TX_200820_01_E02,WMB-10Xv3-CTXsp,cell,NaN,10Xv3,CTXsp,Snap25-IRES2-Cre;Ai14-537272,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,080 CEA-AAA-BST Six3 Sp9 Gaba,0374 CEA-AAA-BST Six3 Sp9 Gaba_2,1358 CEA-AAA-BST Six3 Sp9 Gaba_2,#FF3358,#450099,#990091,#57662E,#662E4F,17,#994817
CATAGACAGTCTAACC-299_B02,CATAGACAGTCTAACC,299_B02,L8TX_200723_01_G09,WMB-10Xv3-CTXsp,cell,NaN,10Xv3,CTXsp,Snap25-IRES2-Cre;Ai14-532618,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,080 CEA-AAA-BST Six3 Sp9 Gaba,0374 CEA-AAA-BST Six3 Sp9 Gaba_2,1358 CEA-AAA-BST Six3 Sp9 Gaba_2,#FF3358,#450099,#990091,#57662E,#662E4F,17,#994817
GAGTTACCATGACGAG-299_A02,GAGTTACCATGACGAG,299_A02,L8TX_200723_01_F09,WMB-10Xv3-CTXsp,cell,NaN,10Xv3,CTXsp,Snap25-IRES2-Cre;Ai14-532618,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,080 CEA-AAA-BST Six3 Sp9 Gaba,0374 CEA-AAA-BST Six3 Sp9 Gaba_2,1358 CEA-AAA-BST Six3 Sp9 Gaba_2,#FF3358,#450099,#990091,#57662E,#662E4F,17,#994817


In [10]:
# Saving to file
save_path = '/data/scRNA/ABCA/AIBS/AWS/expression_matrices/WMB-10Xv3/20230630/WMB-10Xv3-CTXsp-log2-wmeta.h5ad'
expression_data.write_h5ad(save_path)

**Loading the mtx data**

In [2]:
import scanpy as sc
mtx_path = '/data/scRNA/ABCA/AIBS/Direct'
adata_mtx = sc.read_10x_mtx(mtx_path, make_unique=True)
adata_mtx

AnnData object with n_obs × n_vars = 176584 × 31053
    var: 'gene_ids', 'feature_types'